In [1]:
from pydataset import data
import pandas as pd

### 1. Load the mpg dataset. Read the documentation for it, and use the data to answer these questions:

In [ ]:
mpg = data("mpg")
mpg.columns

#### * On average, which manufacturer has the best miles per gallon?

In [ ]:
mpg.groupby("manufacturer")[['cty', 'hwy']].agg("mean").nlargest(1, ['cty', 'hwy'], keep = "all")

### * How many different manufacturers are there?

In [ ]:
print(f"There are {len(mpg.groupby('manufacturer'))} manufacturers.")

#### * How many different models are there?

In [ ]:
print(f"There are {len(mpg.groupby('model'))} models.")

#### * Do automatic or manual cars have better miles per gallon?

In [ ]:
def assign_transmission(trans):
    if trans.find("auto") > -1:
        return "auto"
    else:
        return "manual"

mpg['auto_or_manual'] = mpg.trans.apply(assign_transmission)
mpg.groupby('auto_or_manual')[['cty', 'hwy']].agg("mean")

### 2. Joining and Merging

In [ ]:
import numpy as np
users = pd.DataFrame({
    'id': [1, 2, 3, 4, 5, 6],
    'name': ['bob', 'joe', 'sally', 'adam', 'jane', 'mike'],
    'role_id': [1, 2, 3, 3, np.nan, np.nan]
})
users

In [ ]:
roles = pd.DataFrame({
    'id': [1, 2, 3, 4],
    'name': ['admin', 'author', 'reviewer', 'commenter']
})
roles

In [ ]:
# Right join would have all the roles whether or not there is a match
pd.merge(users, roles, left_on = "role_id", right_on = "id", how = "right")

In [ ]:
# Outer join would return results from both tables whether or not there is a match
pd.merge(users, roles, left_on = "role_id", right_on = "id", how = "outer")

In [ ]:
# If you drop the foreign keys you will not be able to join the tables
users.drop(columns = ['role_id'], inplace = True)
roles.drop(columns = ['id'], inplace = True)
pd.merge(users, roles, left_on = "role_id", right_on = "id", how = "inner")

### 3. Getting data from SQL databases

#### * Create a function named get_db_url. It should accept a username, hostname, password, and database name and return a url formatted like in the examples in this lesson.

In [2]:
def get_db_url(user, host, password, db_name):
    return f"mysql+pymysql://{user}:{password}@{host}/{db_name}"

#### * Use your function to obtain a connection to the employees database.

In [3]:
from env import user, host, password

In [ ]:
url = get_db_url(user, host, password, "employees")
query = """
    SELECT *
        FROM employees
            JOIN dept_emp USING(emp_no)
        WHERE to_date > now();
"""

pd.read_sql(query, url)

#### * Intentionally make a typo in the database url. What kind of error message do you see?

In [ ]:
pd.read_sql(query, url + "typo")

#### * Intentionally make an error in your SQL query. What does the error message look like?

In [ ]:
pd.read_sql(query + "typo", url)

#### * Read the employees and titles tables into two separate dataframes

In [ ]:
employees = pd.read_sql(query, url)
employees

In [ ]:
titles = pd.read_sql("SELECT * FROM titles;", url)
titles

#### * Visualize the number of employees with each title.

In [ ]:
titles.groupby('title').emp_no.agg("count")

In [ ]:
titles.groupby('title').emp_no.agg("count").plot.barh()

#### * Join the employees and titles dataframes together.

In [ ]:
emp_and_title = pd.merge(employees, titles, left_on = "emp_no", right_on = "emp_no")

#### * Visualize how frequently employees change titles.

In [ ]:
emp_and_title.groupby('emp_no').title.agg("count")

In [ ]:
emp_and_title.groupby('emp_no').title.agg("count").head(20).plot.bar(ylabel = "titles held")

#### * For each title, find the hire date of the employee that was hired most recently with that title.

In [ ]:
emp_and_title.groupby('title').hire_date.agg("max")

#### * Write the code necessary to create a cross tabulation of the number of titles by department.

In [ ]:
dept_query = """SELECT *
                    FROM departments;"""
departments = pd.read_sql(dept_query, url)
emp_title_dept = pd.merge(emp_and_title, departments, left_on = "dept_no", right_on = "dept_no")
emp_title_dept

In [ ]:
pd.crosstab(emp_title_dept.title, emp_title_dept.dept_name)

### 4. Use your get_db_url function to help you explore the data from the chipotle database. Use the data to answer the following questions:

In [ ]:
chipotle_url = get_db_url(user, host, password, "chipotle")
chipotle_query = """SELECT *
                        FROM orders"""

chipotle = pd.read_sql(chipotle_query, chipotle_url)
chipotle

#### * What is the total price for each order?

In [ ]:
def convert_to_float(price):
    price = price.replace("$", "")
    return float(price)

chipotle['float_price'] = chipotle.item_price.apply(convert_to_float)
chipotle.groupby('order_id').float_price.agg("sum")

#### * What are the most popular 3 items?

In [ ]:
chipotle.groupby('item_name').quantity.agg("count").sort_values(ascending = False).head(3)

#### * Which item has produced the most revenue?

In [ ]:
chipotle.groupby('item_name').float_price.agg("sum").sort_values(ascending = False).head(1)

### Bonus:  For even more practice with pandas, you can do the exercises from the SQL module, but instead of using SQL to do the aggregation, sorting, joining, etc, use pandas. That is, read the data from all of the tables into pandas dataframes and manipulate the dataframes.

In [4]:
world_url = get_db_url(user, host, password, "world")
world_query = """SELECT *
                    FROM city;"""

cities = pd.read_sql(world_query, world_url)
cities

,ID,Name,CountryCode,District,Population
0,1,Kabul,AFG,Kabol,1780000
1,2,Qandahar,AFG,Qandahar,237500
2,3,Herat,AFG,Herat,186800
3,4,Mazar-e-Sharif,AFG,Balkh,127800
4,5,Amsterdam,NLD,Noord-Holland,731200
...,...,...,...,...,...
4074,4075,Khan Yunis,PSE,Khan Yunis,123175
4075,4076,Hebron,PSE,Hebron,119401
4076,4077,Jabaliya,PSE,North Gaza,113901
4077,4078,Nablus,PSE,Nablus,100231


#### * What languages are spoken in Santa Monica?

In [5]:
lang_query = """SELECT *
                    FROM countrylanguage;"""
languages = pd.read_sql(lang_query, world_url)
languages

,CountryCode,Language,IsOfficial,Percentage
0,ABW,Dutch,T,5.3
1,ABW,English,F,9.5
2,ABW,Papiamento,F,76.7
3,ABW,Spanish,F,7.4
4,AFG,Balochi,F,0.9
...,...,...,...,...
979,ZMB,Tongan,F,11.0
980,ZWE,English,T,2.2
981,ZWE,Ndebele,F,16.2
982,ZWE,Nyanja,F,2.2


In [ ]:
cities_languages = pd.merge(cities, languages, left_on = "CountryCode", right_on = "CountryCode")
language_count = cities_languages.groupby('Name').Language.agg("count")
language_count["Santa Monica"]
# 12 languages spoken in Santa Monica

#### * How many different countries are in each region?

In [7]:
country_query = """SELECT *
                        FROM country"""
countries = pd.read_sql(country_query, world_url) 
countries

,Code,Name,Continent,Region,SurfaceArea,IndepYear,Population,LifeExpectancy,GNP,GNPOld,LocalName,GovernmentForm,HeadOfState,Capital,Code2
0,ABW,Aruba,North America,Caribbean,193.0,NaN,103000,78.4,828.0,793.0,Aruba,Nonmetropolitan Territory of The Netherlands,Beatrix,129.0,AW
1,AFG,Afghanistan,Asia,Southern and Central Asia,652090.0,1919.0,22720000,45.9,5976.0,NaN,Afganistan/Afqanestan,Islamic Emirate,Mohammad Omar,1.0,AF
2,AGO,Angola,Africa,Central Africa,1246700.0,1975.0,12878000,38.3,6648.0,7984.0,Angola,Republic,José Eduardo dos Santos,56.0,AO
3,AIA,Anguilla,North America,Caribbean,96.0,NaN,8000,76.1,63.2,NaN,Anguilla,Dependent Territory of the UK,Elisabeth II,62.0,AI
4,ALB,Albania,Europe,Southern Europe,28748.0,1912.0,3401200,71.6,3205.0,2500.0,Shqipëria,Republic,Rexhep Mejdani,34.0,AL
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
234,YEM,Yemen,Asia,Middle East,527968.0,1918.0,18112000,59.8,6041.0,5729.0,Al-Yaman,Republic,Ali Abdallah Salih,1780.0,YE
235,YUG,Yugoslavia,Europe,Southern Europe,102173.0,1918.0,10640000,72.4,17000.0,NaN,Jugoslavija,Federal Republic,Vojislav Koštunica,1792.0,YU
236,ZAF,South Africa,Africa,Southern Africa,1221037.0,1910.0,40377000,51.1,116729.0,129092.0,South Africa,Republic,Thabo Mbeki,716.0,ZA
237,ZMB,Zambia,Africa,Eastern Africa,752618.0,1964.0,9169000,37.2,3377.0,3922.0,Zambia,Republic,Frederick Chiluba,3162.0,ZM
